In [25]:
import os
from dotenv import load_dotenv

In [26]:
import json
import pandas as pd
import traceback

In [35]:
from langchain.chat_models import ChatOpenAI

In [36]:
load_dotenv()

True

In [37]:
KEY = os.getenv("OPENAI_API_KEY")

In [53]:
llm=ChatOpenAI(openai_api_key = KEY, model_name = "gpt-3.5-turbo",temperature=0.5)

In [54]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback #IMPORTANT
import PyPDF2

In [57]:
RESPONSE_JSON={
   "songs": [
        {
            "title": "Song Title 1",
            "movie": "Movie Name 1",
            "year": 2000
        },
        {
            "title": "Song Title 2",
            "movie": "Movie Name 2",
            "year": 2001
        }
    
   ]
}

In [58]:
TEMPLATE="""
You are an expert in musical knowledge. Given the above text, it is your job to\
create a list of {number} songs for {mood} mood from bollywood since the year 2000.
Make sure that the songs are not repeated and check all the songs to be conforming the mood.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.
Ensure to add {number} of songs
###RESPONSE_JSON
{response_json}

"""

In [59]:
music_generation_prompt=PromptTemplate(
    input_variables = ["number","mood","response_json"],
    template = TEMPLATE
)


In [60]:
music_chain=LLMChain(llm=llm, prompt=music_generation_prompt,output_key="music",verbose=True)

In [61]:
TEMPLATE2 ="""
You are an expert in musical knowledge. Given the list of songs below and the specified mood,\
evaluate whether each song conforms to the {mood} mood.
Provide a detailed analysis for each song, including whether it fits the mood and why.
####Songs_List:
{music}

check from an expert in music:
"""

In [62]:
music_evaluation_prompt = PromptTemplate(input_variables=["mood","music"],template = TEMPLATE2)

In [63]:
review_chain = LLMChain(llm=llm, prompt=music_evaluation_prompt, output_key="review",verbose = True)

In [64]:
generate_evaluate_chain = SequentialChain(chains=[music_chain, review_chain],input_variables=["number","mood","response_json"],
                                          output_variables=["music","review"],verbose=True)

In [65]:
json.dumps(RESPONSE_JSON)

'{"songs": [{"title": "Song Title 1", "movie": "Movie Name 1", "year": 2000}, {"title": "Song Title 2", "movie": "Movie Name 2", "year": 2001}]}'

In [66]:
NUMBER = 10
MOOD = "happy"


In [67]:
###setup token usage
with get_openai_callback() as cb:
    response = generate_evaluate_chain(
    {
       "number":NUMBER,
       "mood":MOOD,
       "response_json":json.dumps(RESPONSE_JSON)
    }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert in musical knowledge. Given the above text, it is your job tocreate a list of 10 songs for happy mood from bollywood since the year 2000.
Make sure that the songs are not repeated and check all the songs to be conforming the mood.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.
Ensure to add 10 of songs
###RESPONSE_JSON
{"songs": [{"title": "Song Title 1", "movie": "Movie Name 1", "year": 2000}, {"title": "Song Title 2", "movie": "Movie Name 2", "year": 2001}]}



> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are an expert in musical knowledge. Given the list of songs below and the specified mood,evaluate whether each song conforms to the happy mood.
Provide a detailed analysis for each song, including whether it fits the mood and why.
####Songs_List:
###RESPONSE_JSON
{"songs": [
{"title": "Koi Mi

In [68]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1655
Prompt Tokens:520
Completion Tokens:1135
Total Cost:0.0030499999999999998


In [81]:
music=response.get('music')

'###RESPONSE_JSON\n{"songs": [\n{"title": "Koi Mil Gaya", "movie": "Kuch Kuch Hota Hai", "year": 1998},\n{"title": "Kabhi Khushi Kabhie Gham", "movie": "Kabhi Khushi Kabhie Gham", "year": 2001},\n{"title": "Balle Balle", "movie": "Bride and Prejudice", "year": 2004},\n{"title": "London Thumakda", "movie": "Queen", "year": 2014},\n{"title": "Gallan Goodiyaan", "movie": "Dil Dhadakne Do", "year": 2015},\n{"title": "Nashe Si Chadh Gayi", "movie": "Befikre", "year": 2016},\n{"title": "Dil Dhadakne Do", "movie": "Zindagi Na Milegi Dobara", "year": 2011},\n{"title": "Ainvayi Ainvayi", "movie": "Band Baaja Baaraat", "year": 2010},\n{"title": "Badtameez Dil", "movie": "Yeh Jawaani Hai Deewani", "year": 2013},\n{"title": "Aankh Marey", "movie": "Simmba", "year": 2018}\n]}'

In [83]:
json_start_index = music.find('{')

In [84]:
json_start_index

17

In [85]:
cleaned_json_str = music[json_start_index:]

In [86]:
cleaned_json_str

'{"songs": [\n{"title": "Koi Mil Gaya", "movie": "Kuch Kuch Hota Hai", "year": 1998},\n{"title": "Kabhi Khushi Kabhie Gham", "movie": "Kabhi Khushi Kabhie Gham", "year": 2001},\n{"title": "Balle Balle", "movie": "Bride and Prejudice", "year": 2004},\n{"title": "London Thumakda", "movie": "Queen", "year": 2014},\n{"title": "Gallan Goodiyaan", "movie": "Dil Dhadakne Do", "year": 2015},\n{"title": "Nashe Si Chadh Gayi", "movie": "Befikre", "year": 2016},\n{"title": "Dil Dhadakne Do", "movie": "Zindagi Na Milegi Dobara", "year": 2011},\n{"title": "Ainvayi Ainvayi", "movie": "Band Baaja Baaraat", "year": 2010},\n{"title": "Badtameez Dil", "movie": "Yeh Jawaani Hai Deewani", "year": 2013},\n{"title": "Aankh Marey", "movie": "Simmba", "year": 2018}\n]}'

In [88]:
parsed_json = json.loads(cleaned_json_str)

In [89]:
parsed_json

{'songs': [{'title': 'Koi Mil Gaya',
   'movie': 'Kuch Kuch Hota Hai',
   'year': 1998},
  {'title': 'Kabhi Khushi Kabhie Gham',
   'movie': 'Kabhi Khushi Kabhie Gham',
   'year': 2001},
  {'title': 'Balle Balle', 'movie': 'Bride and Prejudice', 'year': 2004},
  {'title': 'London Thumakda', 'movie': 'Queen', 'year': 2014},
  {'title': 'Gallan Goodiyaan', 'movie': 'Dil Dhadakne Do', 'year': 2015},
  {'title': 'Nashe Si Chadh Gayi', 'movie': 'Befikre', 'year': 2016},
  {'title': 'Dil Dhadakne Do',
   'movie': 'Zindagi Na Milegi Dobara',
   'year': 2011},
  {'title': 'Ainvayi Ainvayi', 'movie': 'Band Baaja Baaraat', 'year': 2010},
  {'title': 'Badtameez Dil', 'movie': 'Yeh Jawaani Hai Deewani', 'year': 2013},
  {'title': 'Aankh Marey', 'movie': 'Simmba', 'year': 2018}]}

In [94]:
music_table_data= []
for key in parsed_json["songs"]:
    title = key["title"]
    movie = key["movie"]
    year = key["year"]
    music_table_data.append({"Title": title, "Movie": movie, "Year": year})

formatted_music_list = " | ".join([f"Title: {song['Title']}, Movie: {song['Movie']}, Year: {song['Year']}" for song in music_table_data])
print("Music Table Data:")
print(formatted_music_list)

Music Table Data:
Title: Koi Mil Gaya, Movie: Kuch Kuch Hota Hai, Year: 1998 | Title: Kabhi Khushi Kabhie Gham, Movie: Kabhi Khushi Kabhie Gham, Year: 2001 | Title: Balle Balle, Movie: Bride and Prejudice, Year: 2004 | Title: London Thumakda, Movie: Queen, Year: 2014 | Title: Gallan Goodiyaan, Movie: Dil Dhadakne Do, Year: 2015 | Title: Nashe Si Chadh Gayi, Movie: Befikre, Year: 2016 | Title: Dil Dhadakne Do, Movie: Zindagi Na Milegi Dobara, Year: 2011 | Title: Ainvayi Ainvayi, Movie: Band Baaja Baaraat, Year: 2010 | Title: Badtameez Dil, Movie: Yeh Jawaani Hai Deewani, Year: 2013 | Title: Aankh Marey, Movie: Simmba, Year: 2018


In [96]:
music = pd.DataFrame(music_table_data)

In [97]:
music.to_csv("music_generation.csv",index=False)